In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from model import *

unlabel_prob = 0.95 # probability of labels to be unlabeled
percentile = 0     # when a graph is not given, construct a graph 
                    # dropping nodes under this percentile 

sigma = 1.5           # sigma hyperparameter for RBF kernel

hard = False


%matplotlib inline

### Get Data

In [2]:
true_labels, features, graph = Data.Data.load_cora()

In [3]:
labels, is_labeled, labeled_indices, unlabeled_indices = utils.random_unlabel(true_labels,unlabel_prob=unlabel_prob,seed=1)
solution  = true_labels[unlabeled_indices]
weights, graph   = utils.rbf_kernel(features,s=sigma,G=graph,percentile=percentile)
num_nodes = len(labels)
data, validation_data = Data.Data.prepare(labels,labeled_indices,true_labels)

graph given


In [4]:
baseline_prediction = np.rint(labels[unlabeled_indices])
print("[Baseline]　Accuracy:", utils.accuracy(solution, baseline_prediction), 
      "RMSE:", utils.mse(solution, baseline_prediction, num_classes))

TypeError: mse() missing 1 required positional argument: 'num_classes'

### Label Propagation

In [ ]:
lp = LP.LP()
lp_closed_prediction = np.rint(lp.closed(labels,
                                  weights,
                                  labeled_indices,
                                  unlabeled_indices))
# lp_iter_prediction = np.rint(lp.iter(labels,
#                                   weights,
#                                   is_labeled,
#                                   num_iter=100))

In [5]:
print("[Closed]　Accuracy:", utils.accuracy(solution, lp_closed_prediction), 
      "MSE:", utils.mse(solution, lp_closed_prediction, num_classes))
# print("[Iter]　Accuracy:", utils.accuracy(solution, lp_iter_prediction[unlabeled_indices]), 
#       "MSE:", utils.mse(solution, lp_iter_prediction[unlabeled_indices]))

NameError: name 'lp_closed_prediction' is not defined

In [12]:
lp.closed(labels,
                                  weights,
                                  labeled_indices,
                                  unlabeled_indices)

array([ 0.23367431,  0.23445003,  0.23276246, ...,  0.23413873,
        0.23575643,  0.24696987])

In [11]:
# for num_iter in [1,2,3,4,5,10,100,1000,2000]:
#     print(num_iter)
#     lp_iter_prediction = np.rint(lp.iter(labels,
#                                   weights,
#                                   is_labeled,
#                                   num_iter=num_iter))
#     print("Accuracy:", utils.accuracy(solution,lp_iter_prediction[unlabeled_indices]))
#     print("RMSE:", utils.rmse(solution,lp_iter_prediction[unlabeled_indices]))

### DeepLP

In [12]:
dlp = DeepLP.DeepLP(num_nodes,weights,num_iter=100,lr=0.1)
pred, _ = dlp.labelprop(validation_data)
dlp_prediction = np.rint(pred).reshape(num_nodes)[unlabeled_indices]
print("[DeepLP]　Accuracy:", utils.accuracy(solution, dlp_prediction), 
      "RMSE:", utils.rmse(solution, dlp_prediction))


{'loss': 0.20985664, 'labeled_loss': 1.0005437e-10, 'accuracy': 0.70466101, 'true_loss': 0.20985664, 'true_accuracy': 0.70466101}
[DeepLP]　Accuracy: 0.704661016949 RMSE: 0.295338983051


### DeepLP_RBF

In [13]:
dlp_rbf = DeepLP_RBF.DeepLP_RBF(num_nodes, features, graph, sigma, num_iter=100, lr=0.01, loss_type='log',graph_sparse=True) # usually 1

In [14]:
dlp_rbf_prediction,_ = dlp_rbf.labelprop(validation_data)

{'loss': 0.61125052, 'labeled_loss': 1.001363e-05, 'accuracy': 0.70466101, 'true_loss': 0.61125052, 'true_accuracy': 0.70466101}


In [15]:
dlp_rbf.train(data,validation_data,200)

KeyboardInterrupt: 

In [14]:
# dlp_rbf.plot()

# DeepLP_WeightedRBF

In [16]:
theta = np.ones(features.shape[1]).reshape(1,features.shape[1])
dlp_wrbf = DeepLP_WeightedRBF.DeepLP_WeightedRBF(100, num_nodes, features, graph, 1, theta, lr = 0.1, regularize = 0.1)
dlp_wrbf_prediction,_ = dlp_wrbf.labelprop(validation_data)


{'loss': 0.13427219, 'labeled_loss': 0.0, 'accuracy': 0.87203389, 'true_loss': 0.13427219, 'true_accuracy': 0.87203389}


In [17]:
# dlp_wrbf.train(data,validation_data,200)

In [18]:
# dlp_wrbf.plot()

In [5]:
reslp = []
i=0
for x,masked,labeled in zip(data['X'],data['masked'],data['labeled']):
    lp = LP.LP()
    labeled=labeled.reshape(1,2485)
    masked=masked.reshape(1,2485)
    is_labeled=is_labeled.reshape(1,2485)
    x=x.reshape(1,2485)
    labeled_new = is_labeled * labeled
    
    lp_iter_prediction = lp.iter(x,
                                      weights,
                                      labeled_new,
                                      num_iter=100)
    
    reslp.append(np.sum(lp_iter_prediction * masked))
    i+=1
    print(i)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125


In [21]:
accuracies = []
rmses = []
thetas = []

for i in range(100): 
    theta_ = np.random.uniform(0.5,4,features.shape[1])
    if i % 10 == 0:
        print(i)
    dlp_wrbf = DeepLP_WeightedRBF.DeepLP_WeightedRBF(100, num_nodes, features, graph, sigma, theta_, 1, regularize=0)

    dlp_wrbf_prediction = np.rint(dlp_wrbf.labelprop(validation_data,theta_)).reshape(num_nodes)[unlabeled_indices]
    accuracy = utils.accuracy(solution, dlp_wrbf_prediction)
    rmse = utils.rmse(solution, dlp_wrbf_prediction)
    accuracies.append(accuracy)
    rmses.append(rmse)
  
    print(theta_)
    print(accuracy)

0
hi
[ 2.02628044  1.19450436  2.57404266  0.96248108  3.06456589  2.71033147
  2.46038856  1.94946962  3.00232759  3.42431212  1.84317861  3.48630929
  2.56762759  1.63096131  2.74101107  3.02932748  1.64176901  2.15518114
  3.49563196]
0.844491525424
hi
[ 1.63771657  1.80822701  3.92800206  1.64847068  1.1071544   2.33827566
  3.58681737  1.20495782  1.04006146  3.37578736  2.92288964  2.04798005
  3.06065114  1.43610955  2.09173424  0.59781255  0.82008608  1.71243731
  2.59884328]
0.841949152542
hi
[ 3.90943552  1.73965757  3.68955532  3.72284978  1.33334503  2.18024058
  3.2774972   1.67303502  1.01951588  3.47232145  3.08663811  3.01407815
  2.22722414  1.10678607  2.55053819  3.00973293  1.38884229  1.55755151
  3.83865509]
0.851271186441
hi
[ 2.28063716  3.37116237  3.19164961  3.87481998  1.92550923  2.33911803
  3.70932432  3.06925816  1.34350713  1.77447154  2.63138941  1.70803587
  3.41494771  1.73380821  1.20447592  2.31934395  3.93757738  2.82139935
  1.66842561]
0.8584745

KeyboardInterrupt: 